In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/wisesight-sentiment"))

# Any results you write to the current directory are saved as output.

['test.txt', 'train_label.txt', 'train.txt', 'test_majority.csv']


In [2]:
train_text = open("../input/wisesight-sentiment/train.txt", "r")
train_label_text = open("../input/wisesight-sentiment/train_label.txt", "r")
test_text = open("../input/wisesight-sentiment/test.txt", "r")

In [3]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
from IPython.display import display

import matplotlib.pyplot as plt

In [4]:
!pip install pythainlp

    100% |████████████████████████████████| 10.3MB 4.1MB/s eta 0:00:01


In [5]:
sentence = []
label = []
test = []
for line in train_text:
    sentence.append(line.strip())
for line in train_label_text:
    label.append(line.strip())
for line in test_text:
    test.append(line.strip())

In [6]:
import pandas as pd
data = {'sentence':sentence,'label':label}
df = pd.DataFrame(data)
print (df)
data_test = {'test':test}
df_test = pd.DataFrame(test)

                                                sentence label
0      10 ประโยชน์ของฮับมะกอก เกรด A จากตุรกี 1. ช่วย...   neu
1      ขับอัลติส1.8/141ม้า บางทีเจอกะบะกะพริบไฟใร่ตาม...   neg
2      อยู่แบบนี้กันไปนานๆนะ อยู่เป็นเพื่อนในเหงา อยู...   neu
3                             ข้างหลังเลียนแบบ mg3ชัดชัด   neu
4           ,, อยาก ชีวิตดีกับ BLEND285 จัง เลยจ๊ะพี่จ๋า   neu
5      ไม่กล้ากินแล้วอ่ะ กลัววววววตาย แถมการชี้แจงจะอ...   neu
6      ***** มาเพิ่มค่ะ หมดแล้วหมดเลยนะคะ*** Bvlgari ...   neu
7                                              ครูจ๋าาาา   neu
8                 สามารถสั่งจองที่สาขาใกล้บ้านได้เลยนะคะ   neu
9      ก็น่าจะอร่อยนะ ดูจากรูปแล้ว เนื้อน่ากินสุดๆ เห...   pos
10                  คิดถึง Hot Pot Value ไม่จำกัดเวลา...   neu
11                         เอาที่แกสบายใจมีเบียร์ช้างใหม   neu
12     ตอบโจทย์แห่งสังคมยุคใหม่ที่มาพร้อมเทคโลโลยี Sm...   neu
13             จะเป็นอย่างไรเมื่อ Honda มาทำเครื่องดีเซล   neu
14     นี่เป็นคนขับรถเร็วเวลาอยู่คนเดียว ยิ่งเป็นทางท..

In [7]:
from pythainlp.tokenize import word_tokenize
a = 'ฉันรักภาษาไทยเพราะฉันเป็นคนไทย' # u'ฉันรักภาษาไทยเพราะฉันเป็นคนไทย' ใน Python 2.7
b = word_tokenize(a)
print(b) # ['ฉัน', 'รัก', 'ภาษาไทย', 'เพราะ', 'ฉัน', 'เป็น', 'คนไทย']

['ฉัน', 'รัก', 'ภาษาไทย', 'เพราะ', 'ฉัน', 'เป็น', 'คนไทย']


In [8]:
data_df = df
data_df['clean_label']=data_df['label'].str.lower().copy()
data_df.drop('label', axis=1, inplace=True)
data_df.columns = ['Input','Label']
data_df = data_df.drop_duplicates("Input", keep="first")
display(data_df.describe())

,Input,Label
count,24183,24183
unique,24183,4
top,น้องอยากกิน พาน้องไปกินหน่อย,neu
freq,1,14163


In [9]:
data = np.array(data_df.as_matrix(), copy=True)
data_test = np.array(df_test.as_matrix(), copy=True)

action_unique_label = data_df.Label.unique()

action_label_2_num_map = dict(zip(action_unique_label, range(len(action_unique_label))))
action_num_2_label_map = dict(zip(range(len(action_unique_label)), action_unique_label))

print("Create Mappings")
display(action_label_2_num_map)
display(action_num_2_label_map)

print("Before Mappings")
display(data[:, 1])
data[:,1] = np.vectorize(action_label_2_num_map.get)(data[:,1])

print("After Mappings")
display(data[:, 1])


Create Mappings


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


{'neu': 0, 'neg': 1, 'pos': 2, 'q': 3}

{0: 'neu', 1: 'neg', 2: 'pos', 3: 'q'}

Before Mappings


array(['neu', 'neg', 'neu', ..., 'neg', 'neg', 'neu'], dtype=object)

After Mappings


array([0, 1, 0, ..., 1, 1, 0], dtype=object)

In [11]:
all_word = []
for x in sentence:
    b = word_tokenize(x)
    for i in b:
        all_word.append(i)

In [12]:
all_word_uniq = np.unique(all_word)

all_word_final = []
str_char, str_char_count = np.unique(all_word, return_counts=True)
for char, count in zip(str_char, str_char_count):
    if count >= 5:
        all_word_final.append(char)
print(len(all_word_final))
all_word_final = np.array(list(all_word_final))
all_word_final = np.unique(all_word_final)
sorted(all_word_final)
word_map = dict(zip(all_word_final, range(len(all_word_final))))

8322


In [13]:
def count_word(string):
    global all_char, char_map
    result = np.zeros(len(all_word_final))
    b = word_tokenize(string)
    np_str = np.array(b)
    str_char, str_char_count = np.unique(np_str, return_counts=True)
    for char, count in zip(str_char, str_char_count):
        try:
            result[word_map[char]] = count
#             print(count,char)
        except:
            aaa = 5
    return result

# run example feature transformation
print("Example String to feature conversion")
display(data[0, 0])
display(count_word(data[0, 0]))

Example String to feature conversion


'10 ประโยชน์ของฮับมะกอก เกรด A จากตุรกี 1. ช่วยรักษาโรคเบาหวานและความดัน 2. ช่วยป้องกัน และรักษาโรคหัวใจ 3. ช่วยป้องกัน และรักษาโรคมะเร็ง 4. ช่วยบำรุงผิวพรรณแบบ Inside-Out 5. ช่วยบำรุงสมอง และป้องกันโรคอัลไซเมอร์ 6. ช่วยละลายไขมันในร่างกาย 7. ช่วยกำจัดแบคทีเรียและเชื้อราในร่างกาย 8. ช่วยรักษาโรคในท้อง เช่นท้องเสีย หรืออาหารเป็นพิษ 9. ช่วยบรรเทาอาการปวดเมื่อย และรักษาโรคเหน็บชา 10. ช่วยรักษาอาการภูมิแพ้ และโรคไซนัส'

array([32.,  0.,  0., ...,  0.,  0.,  0.])

In [14]:
temp = np.vectorize(count_word, otypes=[object])(data[:, 0])
x_f1 = np.array([[e for e in sl] for sl in temp.tolist()])

temp = np.vectorize(count_word, otypes=[object])(data_test[:, 0])
x_f1_test = np.array([[e for e in sl] for sl in temp.tolist()])

action_label = data[:, 1]
print("Data")
print("Data shape", x_f1.shape)
print("label shape", action_label.shape)

Data
Data shape (24183, 8322)
label shape (24183,)


In [15]:
from sklearn.model_selection import train_test_split
action_train_f1, action_test_f1 = train_test_split(x_f1, test_size=0.1,random_state=42)
action_train_label, action_test_label = train_test_split(action_label, test_size=0.1,random_state=42)

In [16]:
action_train_label.shape

(21764,)

In [17]:
def onehot_label(label,class_type):
    result = np.zeros(len(data_df.Label.unique()))
    result[int(label)] = 1
    return result

In [18]:
temp = np.vectorize(onehot_label, otypes=[object])(action_train_label,'Label')
action_y_train = np.array([[e for e in sl] for sl in temp.tolist()])
temp = np.vectorize(onehot_label, otypes=[object])(action_test_label,'Label')
action_y_test = np.array([[e for e in sl] for sl in temp.tolist()])

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(action_train_f1.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.50))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.50))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.50))
model.add(Dense(len(data_df.Label.unique())))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


In [20]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
model.fit(action_train_f1,action_y_train, validation_data=(action_test_f1, action_y_test), callbacks=[checkpoint],epochs=10,verbose=2)

Train on 21764 samples, validate on 2419 samples
Epoch 1/10
 - 5s - loss: 0.8230 - acc: 0.6827 - val_loss: 0.6804 - val_acc: 0.7342

Epoch 00001: val_loss improved from inf to 0.68042, saving model to model.h5
Epoch 2/10
 - 4s - loss: 0.5922 - acc: 0.7669 - val_loss: 0.6798 - val_acc: 0.7288

Epoch 00002: val_loss improved from 0.68042 to 0.67976, saving model to model.h5
Epoch 3/10
 - 4s - loss: 0.4391 - acc: 0.8237 - val_loss: 0.7373 - val_acc: 0.7255

Epoch 00003: val_loss did not improve from 0.67976
Epoch 4/10
 - 4s - loss: 0.3079 - acc: 0.8774 - val_loss: 0.8830 - val_acc: 0.7375

Epoch 00004: val_loss did not improve from 0.67976
Epoch 5/10
 - 4s - loss: 0.2244 - acc: 0.9139 - val_loss: 1.0333 - val_acc: 0.7139

Epoch 00005: val_loss did not improve from 0.67976
Epoch 6/10
 - 4s - loss: 0.1729 - acc: 0.9366 - val_loss: 1.1956 - val_acc: 0.7222

Epoch 00006: val_loss did not improve from 0.67976
Epoch 7/10
 - 4s - loss: 0.1473 - acc: 0.9464 - val_loss: 1.2129 - val_acc: 0.7115

E

In [21]:
from keras.models import load_model
model = load_model('model.h5')

from sklearn.metrics import f1_score
y_pred = model.predict(action_test_f1)
y_pred = np.argmax(y_pred,axis = 1)
y_true = np.argmax(action_y_test,axis = 1)
print(f1_score(y_true,y_pred,average='micro'))

0.7288135593220338


In [22]:
y_pred = model.predict(x_f1_test)
y_pred = np.argmax(y_pred,axis = 1)

In [23]:
len(y_pred)

3946

In [24]:
print(action_num_2_label_map)
idx = 1
Id = []
Class = []
for x in y_pred:
    Id.append(idx)
    Class.append(action_num_2_label_map[x])
    idx += 1
data = {'Id':Id,'Class':Class}
df = pd.DataFrame(data)

export_csv = df.to_csv ('export_dataframe.csv', index = None, header=True) 

print (df)

{0: 'neu', 1: 'neg', 2: 'pos', 3: 'q'}
        Id Class
0        1   neu
1        2   neu
2        3   neu
3        4   neu
4        5   neu
5        6   pos
6        7   neu
7        8   neg
8        9   pos
9       10   neg
10      11   neu
11      12   neu
12      13   pos
13      14   neu
14      15   neu
15      16   neu
16      17   neg
17      18   neg
18      19   neu
19      20   neg
20      21   neg
21      22   neu
22      23   neg
23      24   neg
24      25   neu
25      26   neu
26      27   neu
27      28   neu
28      29   neu
29      30   neu
...    ...   ...
3916  3917   pos
3917  3918   neu
3918  3919   neu
3919  3920   neu
3920  3921   neu
3921  3922   neu
3922  3923   neu
3923  3924   neu
3924  3925   neu
3925  3926   neu
3926  3927   neu
3927  3928   neu
3928  3929   neu
3929  3930   neu
3930  3931   neu
3931  3932   neg
3932  3933   neu
3933  3934   neu
3934  3935   neu
3935  3936   neg
3936  3937   neu
3937  3938   neu
3938  3939   neu
3939  3940     q
3940  394